In [225]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA
# from sklearn.preprocessing import Imputer
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.metrics import make_scorer
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import ElasticNet
from sklearn.feature_selection import SelectKBest, chi2, f_regression, mutual_info_regression, RFE
import sklearn
# import tflearn
import tensorflow as tf
import seaborn
import warnings
import os
warnings.filterwarnings('ignore')

# faati test mest
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# do not delete
import random

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

In [226]:
def make_submission(prediction, filename):
    assert type(prediction) == np.ndarray, "Pass a numpy.ndarray"
    assert prediction.shape[0] == 80, f"Missing data points Expected 80 Got {prediction.shape[0]}"
    assert prediction.shape[1] == 595, f"Number of features predicted is not correct Excpected 595 Got {prediction.shape[1]}"
    assert type(filename) == str, "Filename must be a string"
    assert '.csv' in filename, "Add extension '.csv'"
    
    df = pd.DataFrame(prediction.flatten())
    df.index.name = "ID"
    df = df.rename(columns={0:"predicted"})
    df.to_csv(filename)
    print(f"{filename} saved at {os.getcwd()}")
    return df

In [227]:
def remove_id_column(dataframe):
    assert type(dataframe) == pd.core.frame.DataFrame, f"{type(dataframe)} != pd.core.frame.DataFrame"
    return dataframe.drop("ID", 1)


In [257]:
def try_models(train,train_labels,test,test_labels, results):

    def test_model(clf):

        clf.fit(train, train_labels)
        test_predict = clf.predict(test)
        actual = test_labels.flatten()
        predicted = test_predict.flatten()
        scores = mean_squared_error(predicted, actual)

        return scores

    clf = linear_model.LinearRegression()
    test_result = test_model(clf)
    model_name = 'LinearRegression'
    if model_name in results.keys():
        results[model_name].append(test_result)
    else:
        results.update({model_name:[test_result]})
    
    clf = linear_model.Ridge()
    test_result = test_model(clf)
    model_name = 'Ridge'
    if model_name in results.keys():
        results[model_name].append(test_result)
    else:
        results.update({model_name:[test_result]})

    clf = MultiOutputRegressor(SGDRegressor())
    test_result = test_model(clf)
    model_name = 'SGDRegressor'
    if model_name in results.keys():
        results[model_name].append(test_result)
    else:
        results.update({model_name:[test_result]})

    clf = MultiOutputRegressor(linear_model.BayesianRidge())
    test_result = test_model(clf)
    model_name = 'BayesianRidge'
    if model_name in results.keys():
        results[model_name].append(test_result)
    else:
        results.update({model_name:[test_result]})
    
    clf = MultiOutputRegressor(linear_model.HuberRegressor())
    test_result = test_model(clf)
    model_name = 'HuberRegressor'
    if model_name in results.keys():
        results[model_name].append(test_result)
    else:
        results.update({model_name:[test_result]})
    
    clf = linear_model.Lasso(alpha=1e-4)
    test_result = test_model(clf)
    model_name = 'Lasso'
    if model_name in results.keys():
        results[model_name].append(test_result)
    else:
        results.update({model_name:[test_result]})
        
    clf = BaggingRegressor()
    test_result = test_model(clf)
    model_name = 'BaggingRegressor'
    if model_name in results.keys():
        results[model_name].append(test_result)
    else:
        results.update({model_name:[test_result]})

    clf = ElasticNet()
    test_result = test_model(clf)
    model_name = 'ElasticNet'
    if model_name in results.keys():
        results[model_name].append(test_result)
    else:
        results.update({model_name:[test_result]})
    
    clf = RandomForestRegressor()
    test_result = test_model(clf)
    model_name = 'RandomForestRegressor'
    if model_name in results.keys():
        results[model_name].append(test_result)
    else:
        results.update({model_name:[test_result]})

    clf = MultiOutputRegressor(AdaBoostRegressor())
    test_result = test_model(clf)
    model_name = 'AdaBoostRegressor'
    if model_name in results.keys():
        results[model_name].append(test_result)
    else:
        results.update({model_name:[test_result]})
    
    clf = MultiOutputRegressor(svm.SVR(kernel="linear"))
    test_result = test_model(clf)
    model_name = 'SVR(kernel="linear")'
    if model_name in results.keys():
        results[model_name].append(test_result)
    else:
        results.update({model_name:[test_result]})
       
    clf = MultiOutputRegressor(svm.SVR(kernel="rbf"))
    test_result = test_model(clf)
    model_name = 'SVR(kernel="rbf")'
    if model_name in results.keys():
        results[model_name].append(test_result)
    else:
        results.update({model_name:[test_result]})
    
    return results

In [383]:
random.seed(1)
def cv(data, labels, n_fold, model=None, shuffle=False):
    kf = KFold(n_splits=n_fold, shuffle=shuffle)
    results = {}
    if model is not None:
        results['YourModel'] = []
    for train_index, test_index in kf.split(data):
        train = data[train_index, :]
        train_labels = labels[train_index, :]
        test = data[test_index, :]
        test_labels = labels[test_index, :]

        scaler = MinMaxScaler()
        # Fit on training set only.
        scaler.fit(train)
        # Apply transform to both the training set and the test set.
        train = scaler.transform(train)
        test = scaler.transform(test)

        pca = PCA(0.70)
        pca.fit(train)
        train = pca.transform(train)
        test = pca.transform(test)

        if model == None:
            results = try_models(train,train_labels,test,test_labels, results)
        else:
            model.fit(train, train_labels)
            test_predict = model.predict(test)
            actual = test_labels.flatten()
            predicted = test_predict.flatten()
            scores = mean_squared_error(predicted, actual)
            results['YourModel'].append(scores)

    # Add Variance and Mean Values as columns  
    # to the results dataframe
    results_df = pd.DataFrame(results).T
    variance = results_df.var(axis=1)
    mean = results_df.mean(axis=1)
    results_df.insert(5, "Var", variance)
    results_df.insert(6, "Mean", mean)  

    return results_df

In [378]:
def train_model(model, data, labels):
    scaler = MinMaxScaler()
    # Fit on training set only.
    scaler.fit(data)
    # Apply transform to both the training set and the test set.
    data = scaler.transform(data)

    pca = PCA(0.70)
    pca.fit(data)
    train = pca.transform(data)
    print(train.shape)
    model.fit(data, labels)
    return model

In [361]:
X = pd.read_csv('train_t0.csv')
Y = pd.read_csv('train_t1.csv')
test = pd.read_csv('test_t0.csv')
X = remove_id_column(X)
Y = remove_id_column(Y)
test = remove_id_column(test)

$t_0$ and $t_1$ might not be same for all samples thus searching outlier over features is quite meaningless. Lets check their differences.

In [390]:
data = X.to_numpy()
labels = Y.to_numpy()

diff = labels - data
diff_norm = sklearn.preprocessing.normalize(diff, axis=0)

In [391]:
z = np.abs(stats.zscore(diff, 1))
thresh = 4
data_clean = data[(z<thresh).all(axis=1)]
data_clean.shape
labels_clean = data[(z<thresh).all(axis=1)]
labels_clean.shape

clf = linear_model.Ridge()
clf.fit(data_clean, labels_clean)
# clf = train_model(clf, data_clean, labels_clean)
test = pd.read_csv('test_t0.csv')
test = remove_id_column(test)
predict = clf.predict(test)

In [392]:
test_gonderdigim = pd.read_csv('batuyu_yok_ediyorum.csv')
predict = make_submission(predict, 'batuyu_yok_eden_bu_muydu.csv')

batuyu_yok_eden_bu_muydu.csv saved at /content


In [394]:
print(predict.head())
print(test_gonderdigim.head())
print(predict.tail())
print(test_gonderdigim.tail())

    predicted
ID           
0    0.048603
1    0.098349
2    0.181086
3    0.000000
4    0.048603
   ID  predicted
0   0   0.048603
1   1   0.098349
2   2   0.181086
3   3   0.000000
4   4   0.048603
       predicted
ID              
47595   0.129815
47596   0.134323
47597   0.036656
47598   0.036656
47599   0.310462
          ID  predicted
47595  47595   0.129815
47596  47596   0.134323
47597  47597   0.036656
47598  47598   0.036656
47599  47599   0.310462


In [ ]:
for i in range(10):
    plt.hist(diff[:, i], 200)
    plt.show()

In [ ]:
for i in range(10):
    plt.scatter(np.arange(len(diff[:,i])),diff[:, i])
    plt.show()

In [323]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(random_state=0).fit(diff)
k = clf.predict(diff)
print(k.shape)
print(np.sum(k==-1))
data_clean = data[np.where(k==1)]
clf2 = IsolationForest(random_state=0).fit(data)
k2 = clf2.predict(data)
print(np.sum(k2==-1))
clf3 = IsolationForest(random_state=0).fit(labels)
k3 = clf3.predict(labels)
print(np.sum(k3==-1))
anomalies = np.logical_and(k3==1, np.logical_and(k2==1, k==1))
print(anomalies.shape)
locations = np.where(anomalies)
print(len(locations[0]))

(150,)
9
6
6
(150,)
139


In [324]:
data_clean = data[locations]
labels_clean = labels[locations]
print(data_clean.shape)
print(labels_clean.shape)

(139, 595)
(139, 595)


In [327]:
results = cv(data_clean, labels_clean, 5, shuffle=True)

In [328]:
results.head(20)

,0,1,2,3,4,Var,Mean
LinearRegression,0.003022,0.003014,0.002834,0.002689,0.002603,3.557636e-08,0.002833
Ridge,0.002920,0.002896,0.002729,0.002592,0.002525,3.116417e-08,0.002732
SGDRegressor,0.002545,0.002293,0.002229,0.002095,0.002349,2.746103e-08,0.002302
BayesianRidge,0.002327,0.002159,0.002147,0.001993,0.002125,1.417797e-08,0.002150
HuberRegressor,0.003134,0.003130,0.002898,0.002926,0.002725,2.986998e-08,0.002963
Lasso,0.002946,0.002930,0.002760,0.002615,0.002552,3.202109e-08,0.002761
BaggingRegressor,0.002642,0.002352,0.002545,0.002408,0.002491,1.299822e-08,0.002488
ElasticNet,0.002750,0.002499,0.002644,0.002311,0.002650,2.908367e-08,0.002571
RandomForestRegressor,0.002458,0.002237,0.002237,0.002214,0.002292,9.885953e-09,0.002288
AdaBoostRegressor,0.002545,0.002324,0.002298,0.002194,0.002394,1.699822e-08,0.002351


In [277]:
results2 = results.drop("Mean",1 )
results2 = results2.drop("Var", 1)
results2.head(20)


LinearRegression         0.002243
Ridge                    0.002238
SGDRegressor             0.002305
BayesianRidge            0.002154
HuberRegressor           0.002255
Lasso                    0.002236
BaggingRegressor         0.002453
ElasticNet               0.002569
RandomForestRegressor    0.002247
AdaBoostRegressor        0.002334
SVR                      0.003351
SVR(kernel="linear")     0.003291
SVR(kernel="rbf")        0.003351
dtype: float64

In [363]:
from scipy import stats
z = np.abs(stats.zscore(diff, 1))
print(z)
print(np.sum(z>4, axis=1))

[[0.27550312 0.42684763 0.05926019 ... 0.97315076 0.97315076 1.65221842]
 [0.04977388 0.11642169 0.10556146 ... 0.19023382 0.19023382 0.60581282]
 [0.33303279 0.26241309 0.69007624 ... 0.13325332 0.13325332 0.79129724]
 ...
 [0.85393891 0.60745419 1.48217688 ... 0.27721018 0.27721018 1.10369588]
 [0.21128766 0.10509232 0.27422087 ... 0.88766977 0.88766977 1.70621432]
 [0.45676521 0.05742982 0.24308232 ... 0.3140789  0.3140789  0.19442973]]
[2 0 2 3 0 2 0 0 0 1 0 0 0 0 2 0 0 0 1 0 0 0 0 2 0 1 0 1 0 0 1 3 0 3 0 0 2
 1 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 2 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 4 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1 2 1 0 1 0 0 1 0 1 1 0 0 0 1
 1 0 0 0 1 1 0 1 0 2 0 0 1 1 0 2 0 0 3 0 1 0 6 0 1 0 0 1 0 0 2 1 0 2 1 1 0
 2 1]


In [364]:
z = np.abs(stats.zscore(diff, 1))
thresh = 4
data_clean = data[(z<thresh).all(axis=1)]
data_clean.shape
labels_clean = data[(z<thresh).all(axis=1)]
labels_clean.shape

(91, 595)

In [371]:
z_results = cv(data_clean, labels_clean, 5, shuffle=True)

In [372]:
z_results.head(20)

,0,1,2,3,4,Var,Mean
LinearRegression,0.000509,0.000445,0.000579,0.001472,0.000527,1.855842e-07,0.000707
Ridge,0.000513,0.000450,0.000578,0.001474,0.000530,1.848617e-07,0.000709
SGDRegressor,0.001338,0.001136,0.001282,0.002289,0.001186,2.280844e-07,0.001446
BayesianRidge,0.000514,0.000451,0.000573,0.001475,0.000528,1.858500e-07,0.000708
HuberRegressor,0.000533,0.000456,0.000609,0.001529,0.000561,1.988437e-07,0.000738
Lasso,0.000511,0.000450,0.000579,0.001476,0.000530,1.860553e-07,0.000709
BaggingRegressor,0.002236,0.001528,0.002015,0.003856,0.001931,8.098340e-07,0.002313
ElasticNet,0.002531,0.001983,0.002398,0.004356,0.002504,8.498229e-07,0.002754
RandomForestRegressor,0.001889,0.001432,0.001816,0.003715,0.001780,8.196422e-07,0.002126
AdaBoostRegressor,0.001628,0.001251,0.001558,0.003318,0.001487,6.949349e-07,0.001849


In [382]:
clf = RandomForestRegressor()
clf = train_model(clf, data_clean, labels_clean)
test = pd.read_csv('test_t0.csv')
test = remove_id_column(test)
predict = clf.predict(test)

(91, 16)


In [384]:
test_gonderdigim = pd.read_csv('batuyu_yok_ediyorum.csv')
predict = make_submission(predict, 'batuyu_yok_eden_bu_muydu.csv')

batuyu_yok_eden_bu_muydu.csv saved at /content


In [385]:
print(predict.head())
print(test_gonderdigim.head())

    predicted
ID           
0    0.048802
1    0.076645
2    0.054947
3    0.000000
4    0.048802
   ID  predicted
0   0   0.048603
1   1   0.098349
2   2   0.181086
3   3   0.000000
4   4   0.048603


In [340]:
make_submission(predict, "batuyu_yok_ediyorum.csv")

adaboostregressor_z_score_st_45.csv saved at /content


,predicted
ID,
0,0.135403
1,0.101400
2,0.199240
3,0.000000
4,0.120368
...,...
47595,0.145054
47596,0.162142
47597,0.044967


In [69]:
diff_df = pd.DataFrame(diff)
Q1 = diff_df.quantile(0.20)
Q3 = diff_df.quantile(0.80)
IQR = Q3-Q1
print(IQR)

0      0.044522
1      0.055733
2      0.065716
3      0.000000
4      0.044522
         ...   
590    0.061825
591    0.063336
592    0.041889
593    0.041889
594    0.095796
Length: 595, dtype: float64


In [70]:
boston_df_out = diff_df[~((diff_df < (Q1 - 1.5 * IQR)) |(diff_df > (Q3 + 1.5 * IQR))).any(axis=1)]
boston_df_out.shape

(64, 595)

In [233]:
data = X.to_numpy()
labels = Y.to_numpy()

scaler = MinMaxScaler()

# Fit on training set only.
scaler.fit(data)
# Apply transform to both the training set and the test set.
data = scaler.transform(data)
labels = scaler.transform(labels)
test = scaler.transform(test)

In [ ]:
prediction = []
for i in range(X.shape[1]):
    selectkbest = SelectKBest(f_regression, k=20).fit(data, labels[:, i])
    data_new = selectkbest.transform(data)
    model = RandomForestRegressor()
    model.fit(data_new, labels[:, i])
    test_new = selectkbest.transform(test)
    prediction.append(model.predict(test_new))

In [ ]:
predict = np.asarray(prediction).T
predict.shape

(80, 595)

In [ ]:
make_submission(predict, "mayali_orumcek_gozu_v2.csv")

mayali_orumcek_gozu_v2.csv saved at /content


,predicted
ID,
0,0.382457
1,0.209713
2,0.508423
3,0.000000
4,0.398422
...,...
47595,0.539263
47596,0.493942
47597,0.151568


In [ ]:
train = X.to_numpy()
labels = Y.to_numpy()
test_data = test.to_numpy()

diff = train - labels
square = np.square(diff)
sum = np.sum(square, 0)
threshold = 0.01
print(np.where(sum<threshold))
for i in np.where(sum<threshold):
    train = np.delete(train, i, 1)
    test_data = np.delete(test_data, i, 1)
print(train.shape)
print(test_data.shape)


(array([  3, 465, 468, 496, 499, 527]),)
(150, 589)
(80, 589)


In [126]:
test = data[-15:]
test_labels = labels[-15:]
train = data[:-15]
train_labels = labels[:-15]
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(train)
# Apply transform to both the training set and the test set.
train = scaler.transform(train)
# test_data = scaler.transform(test_data)
test = scaler.transform(test)


pca = PCA(0.70)
pca.fit(train)
train = pca.transform(train)
# testdata = pca.transform(test_data)
test = pca.transform(test)
print(train.shape, train_labels.shape)
model = MultiOutputRegressor(AdaBoostRegressor())
# from xgboost import XGBRFRegressor
# model = MultiOutputRegressor(XGBRFRegressor(objective='reg:squarederror', n_estimators=200, verbosity=0))
# parameters = {'n_estimators':[1,2000]}
# model = RandomForestRegressor(n_estimators=200)
# clf = GridSearchCV(model, parameters)
# clf.fit(train, train_labels)
# model = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=34, learning_rate=1.27, max_depth=3))
model.fit(train, train_labels)
prediction = model.predict(test)
print(mean_squared_error(prediction, test_labels))

(126, 20) (126, 595)
0.0026902277685429167


In [ ]:
clf.get_params()

{'cv': None,
 'error_score': nan,
 'estimator': RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False),
 'estimator__bootstrap': True,
 'estimator__ccp_alpha': 0.0,
 'estimator__criterion': 'mse',
 'estimator__max_depth': None,
 'estimator__max_features': 'auto',
 'estimator__max_leaf_nodes': None,
 'estimator__max_samples': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_impurity_split': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estima

In [ ]:
test = scaler.transform(test)
test = pca.transform(test)
prediction = clf.predict(test)
print(mean_squared_error(test_labels, prediction))

0.0024387781369686595


In [ ]:
prediction = model.predict(testdata)
make_submission(prediction, "rf_pca60.csv")

rf_pca60.csv saved at /content


,predicted
ID,
0,0.056785
1,0.091636
2,0.116463
3,0.000000
4,0.056785
...,...
47595,0.133711
47596,0.131512
47597,0.038767


In [ ]:
# pcc_features = []
# corr_features = []
# corr_matrix = X.corr()

# # determining similar (mutually-correlated) features
# for i in range(1, len(corr_matrix.columns)):
#     for j in range(1, i+1):
#         if np.abs(corr_matrix[f"f{i}"][f"f{j}"]) > 0.75:
#             corr_features.append(f"f{i}")

# corr_matrix = corr_matrix.drop(corr_features)  # eliminating similar features
# corr_label = corr_matrix["Labels"].abs()  # taking absolute of correlations

# sorted_corr_label = corr_label.sort_values(na_position="last", ascending=False)
# feature_names = sorted_corr_label.index

# for i in range(1, nof_features + 1): # taking most informative n features
#     pcc_features.append(feature_names[i])

# print(pcc_features)

In [ ]:
corr_matrix.head()

,f4,f466,f469,f497,f500,f528,f595
f1,NaN,NaN,NaN,NaN,NaN,NaN,-0.269706
f2,NaN,NaN,NaN,NaN,NaN,NaN,-0.084309
f3,NaN,NaN,NaN,NaN,NaN,NaN,-0.176075
f4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f5,NaN,NaN,NaN,NaN,NaN,NaN,-0.269706


In [ ]:
model = RandomForestRegressor()
selector = RFE(model, n_features_to_select=20)
selector = selector.fit(train, train_labels)

ValueError: ignored

In [ ]:
def lets_try(train,train_labels,test,test_labels):
    results={}
    
    print(train.shape)
    print(test.shape)
    print(train_labels.shape)
    print(test_labels.shape)
    def test_model(clf):

        clf.fit(train, train_labels)
        test_predict = clf.predict(test)
        scores = mean_squared_error(test_labels, test_predict)

        return [scores]

    clf = linear_model.LinearRegression()
    print(f"Linear Regression = {test_model(clf)}")
    
    clf = linear_model.Ridge()
    print(f"Ridge = {test_model(clf)}")
    
    # clf = MultiOutputRegressor(SGDRegressor())
    # print(f"SGDRegressor = {test_model(clf)}")

    clf = MultiOutputRegressor(linear_model.BayesianRidge())
    print(f"BayesianRidge = {test_model(clf)}")
    
    clf = MultiOutputRegressor(linear_model.HuberRegressor())
    print(f"HuberRegressor= {test_model(clf)}")
    
    clf = linear_model.Lasso(alpha=1e-4)
    print(f"Lasso = {test_model(clf)}")
        
    clf = BaggingRegressor()
    print(f"BaggingRegressor = {test_model(clf)}")

    clf = ElasticNet()
    print(f"ElasticNet = {test_model(clf)}")
    
    clf = RandomForestRegressor()
    print(f"RandomForestRegressor= {test_model(clf)}")
    
    clf = MultiOutputRegressor(AdaBoostRegressor())
    print(f"AdaBoostRegressor = {test_model(clf)}")
    
    clf = MultiOutputRegressor(svm.SVR())
    print(f"SVM RBF = {test_model(clf)}")
    
    clf = MultiOutputRegressor(svm.SVR(kernel="linear"))
    print(f"SVM Linear = {test_model(clf)}")
       
    clf = MultiOutputRegressor(svm.SVR(kernel="rbf"))
    print(f"SVM Linear rbf {test_model(clf)}")


# lets_try(train,train_labels)

In [ ]:
kf = KFold(n_splits=10)
for train_index, test_index in kf.split(X):
    train = X[train_index, :]
    train_labels = Y[train_index, :]
    test = X[test_index, :]
    test_labels = Y[test_index, :]

    # Fit on training set only.
    scaler.fit(train)
    # Apply transform to both the training set and the test set.
    train = scaler.transform(train)
    test = scaler.transform(test)

    pca = PCA(0.70)
    pca.fit(train)
    train = pca.transform(train)
    test = pca.transform(test)

    lets_try(train,train_labels,test,test_labels)


(135, 19)
(15, 19)
(135, 595)
(15, 595)
Linear Regression = [0.005802272956797038]
Ridge = [0.005799988728181303]
BayesianRidge = [0.004297786222005502]
HuberRegressor= [0.004622051656589287]
Lasso = [0.005790250820408533]
BaggingRegressor = [0.004056072586337191]
ElasticNet = [0.0030826790406581098]
RandomForestRegressor= [0.00402007929036715]
AdaBoostRegressor = [0.0041291666017797945]
SVM RBF = [0.0054829324623424285]
SVM Linear = [0.007069600173376448]
SVM Linear rbf [0.0054829324623424285]
(135, 16)
(15, 16)
(135, 595)
(15, 595)
Linear Regression = [0.006744191885778283]
Ridge = [0.006743680477484871]
BayesianRidge = [0.006507214061402251]
HuberRegressor= [0.006800372756951189]
Lasso = [0.006742680791274501]
BaggingRegressor = [0.008102009029265259]
ElasticNet = [0.0069362924667603545]
RandomForestRegressor= [0.010728668527239145]
AdaBoostRegressor = [0.009640671034316697]
SVM RBF = [0.007188838358384941]
SVM Linear = [0.007198396567356369]
SVM Linear rbf [0.007188838358384941]
(1

In [ ]:
test = data[-15:]
test_labels = labels[-15:]
train = data[:-15]
train_labels = labels[:-15]
scaler = StandardScaler()


# Fit on training set only.
scaler.fit(train)
# Apply transform to both the training set and the test set.
train = scaler.transform(train)
test = scaler.transform(test)

In [ ]:
train_new = SelectKBest(f_regression, k=20).fit_transform(train, train_labels)


pca = PCA(0.70)
pca.fit(train)
train = pca.transform(train)
test = pca.transform(test)
train.shape

ValueError: ignored

In [ ]:
 from xgboost import XGBRFRegressor
 model = MultiOutputRegressor(XGBRFRegressor(n_estimators=200, verbosity=0, 
                                             subsmaple=0.4, colsample_bynode=0.2))

In [ ]:
model.fit(train,train_labels)

MultiOutputRegressor(estimator=XGBRFRegressor(base_score=0.5,
                                              colsample_bylevel=1,
                                              colsample_bynode=0.2,
                                              colsample_bytree=1, gamma=0,
                                              learning_rate=1, max_delta_step=0,
                                              max_depth=3, min_child_weight=1,
                                              missing=None, n_estimators=200,
                                              n_jobs=1, nthread=None,
                                              objective='reg:linear',
                                              random_state=0, reg_alpha=0,
                                              reg_lambda=1, scale_pos_weight=1,
                                              seed=None, silent=None,
                                              subsample=0.8, subsmaple=0.4,
                                              ver

In [ ]:
prediction = model.predict(test)
score = mean_squared_error(prediction, test_labels)
print(score)

0.0028515496249414737


In [ ]:
from sklearn.multioutput import MultiOutputRegressor



lets_try(train,train_labels,test,test_labels)

(135, 16)
(15, 16)
(135, 595)
(15, 595)
Linear Regression = [0.0025963257876526254]
Ridge = [0.0025961320992643937]
SGDRegressor = [460818299441.5089]
BayesianRidge = [0.0025376615663518787]
HuberRegressor= [0.0026339961831321882]
Lasso = [0.0025950992079001494]
BaggingRegressor = [0.0028120621037918438]
ElasticNet = [0.0029951336319722984]
RandomForestRegressor= [0.0025045804372139393]
AdaBoostRegressor = [0.0026053968063475216]
SVM RBF = [0.004267637341080627]
SVM Linear = [0.004026565252999204]
SVM Linear rbf [0.004267637341080627]


In [ ]:
test = data[-15:]
test_labels = labels[-15:]
train = data[:-1b5]
train_labels = labels[:-15]
scaler = StandardScaler()


# Fit on training set only.
scaler.fit(train)
# Apply transform to both the training set and the test set.
train = scaler.transform(train)
test = scaler.transform(test)

(150, 596)
(150, 596)


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f556,f557,f558,f559,f560,f561,f562,f563,f564,f565,f566,f567,f568,f569,f570,f571,f572,f573,f574,f575,f576,f577,f578,f579,f580,f581,f582,f583,f584,f585,f586,f587,f588,f589,f590,f591,f592,f593,f594,f595
0,0.193382,0.096121,0.289502,0,0.193382,0.096121,0.273247,0.466629,0.177126,0.273247,0.383924,0.577305,0.287803,0.383924,0.110677,0.279031,0.472412,0.182910,0.279031,0.005784,0.104893,0.110154,0.303536,0.014033,0.110154,0.163093,0.273770,0.168877,0.656734,0.850116,0.560613,0.656734,0.383487,0.272810,0.377703,0.546580,0.048374,0.145008,0.144495,0.048374,...,0.126646,0.048892,0.193999,0.091083,0.063588,0.063588,0.015484,0.208866,0.080636,0.015484,0.257763,0.368439,0.263546,0.094670,0.641250,0.063858,0.251580,0.445395,0.343711,0.533262,0.209793,0.433611,0.070879,0.055528,0.217489,0.052125,0.199823,0.115555,0.533486,0.012033,0.110908,0.193465,0.186630,0.174750,0.096996,0.242103,0.042979,0.015484,0.015484,0.048103
1,0.014410,0.064584,0.050174,0,0.014410,0.064584,0.115367,0.100957,0.050783,0.115367,0.280751,0.266341,0.216167,0.280751,0.165384,0.133898,0.119488,0.069314,0.133898,0.018531,0.146853,0.114678,0.100268,0.050094,0.114678,0.000689,0.166073,0.019220,0.131172,0.116761,0.066588,0.131172,0.015805,0.149580,0.002727,0.016494,0.065569,0.051158,0.000985,0.065569,...,0.131344,0.142796,0.153824,0.143518,0.229512,0.229512,0.013585,0.027995,0.078169,0.013585,0.128952,0.294336,0.147483,0.128263,0.144757,0.079154,0.139870,0.143458,0.182415,0.120655,0.185169,0.062271,0.096190,0.112874,0.141057,0.152346,0.014494,0.144452,0.025750,0.086677,0.101471,0.028694,0.187073,0.111753,0.100301,0.089273,0.099579,0.013585,0.013585,0.243097
2,0.070879,0.077238,0.148117,0,0.070879,0.077238,0.162528,0.233407,0.085291,0.162528,0.086521,0.157399,0.009283,0.086521,0.076008,0.154000,0.224879,0.076762,0.154000,0.008529,0.067479,0.149078,0.219956,0.071840,0.149078,0.013451,0.062557,0.004922,0.197996,0.268875,0.120758,0.197996,0.035468,0.111476,0.043997,0.048919,0.074722,0.145600,0.002516,0.074722,...,0.012306,0.046075,0.071845,0.042282,0.172846,0.172846,0.046046,0.116925,0.031192,0.046046,0.116482,0.040474,0.107954,0.103031,0.151950,0.028675,0.122083,0.183384,0.183365,0.068136,0.083248,0.091681,0.087642,0.117377,0.190595,0.080120,0.082894,0.108819,0.072999,0.055021,0.085059,0.160802,0.142508,0.114494,0.080725,0.054955,0.084518,0.046046,0.046046,0.126800
3,0.012150,0.053557,0.065708,0,0.012150,0.053557,0.135480,0.147630,0.081922,0.135480,0.065623,0.077774,0.012066,0.065623,0.069856,0.150656,0.162806,0.097099,0.150656,0.015176,0.085032,0.105734,0.117884,0.052177,0.105734,0.029746,0.040111,0.044922,0.126453,0.138604,0.072896,0.126453,0.009026,0.060830,0.024202,0.020719,0.071057,0.058907,0.124615,0.071057,...,0.124101,0.156046,0.150542,0.138081,0.225818,0.225818,0.007430,0.019580,0.046128,0.007430,0.128050,0.058194,0.143226,0.098304,0.119024,0.078487,0.101074,0.116838,0.134783,0.144918,0.126458,0.099879,0.075145,0.101797,0.167031,0.028526,0.083871,0.105492,0.008554,0.062910,0.066011,0.034460,0.122039,0.094287,0.062342,0.067846,0.080307,0.007430,0.007430,0.218388
4,0.035510,0.072512,0.037002,0,0.035510,0.072512,0.056819,0.021309,0.015693,0.056819,0.092735,0.057225,0.020223,0.092735,0.035916,0.154670,0.119161,0.082158,0.154670,0.097851,0.061935,0.147432,0.111922,0.074919,0.147432,0.090612,0.054697,0.007239,0.216227,0.180718,0.143715,0.216227,0.159408,0.123492,0.061557,0.068796,0.038396,0.002886,0.034116,0.038396,...,0.136243,0.151124,0.122952,0.177226,0.250665,0.250665,0.017878,0.017631,0.054634,0.017878,0.038941,0.074857,0.136792,0.129553,0.198349,0.020518,0.154999,0.109175,0.104394,0.120631,0.163377,0.066896,0.080816,0.108183,0.137156,0.111283,0.098557,0.122693,0.014264,0.014730,0.098395,0.117918,0.121873,0.096543,0.081662,0.109835,0.055561,0.017878,0.017878,0.232787


In [ ]:
type(train)

pandas.core.frame.DataFrame

In [ ]:
label = label.drop("ID",1)
label.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f556,f557,f558,f559,f560,f561,f562,f563,f564,f565,f566,f567,f568,f569,f570,f571,f572,f573,f574,f575,f576,f577,f578,f579,f580,f581,f582,f583,f584,f585,f586,f587,f588,f589,f590,f591,f592,f593,f594,f595
0,0.100531,0.032178,0.132709,0,0.100531,0.032178,0.155462,0.255994,0.123285,0.155462,0.346908,0.447440,0.314730,0.346908,0.191446,0.144941,0.245472,0.112763,0.144941,0.010521,0.201967,0.111707,0.212238,0.079529,0.111707,0.043756,0.235201,0.033234,0.113329,0.213860,0.081151,0.113329,0.042133,0.233579,0.031612,0.001622,0.006050,0.106581,0.026128,0.006050,...,0.008971,0.065154,0.060051,0.078424,0.162326,0.162326,0.055889,0.044643,0.088066,0.055889,0.211351,0.402797,0.200830,0.167595,0.169218,0.061939,0.196242,0.228053,0.214949,0.199267,0.341571,0.170295,0.147523,0.154134,0.232804,0.179231,0.014141,0.200908,0.054104,0.130524,0.173879,0.241906,0.198942,0.209243,0.153060,0.158163,0.139791,0.055889,0.055889,0.218214
1,0.014826,0.071366,0.056540,0,0.014826,0.071366,0.125061,0.110234,0.053695,0.125061,0.253270,0.238443,0.181904,0.253270,0.128209,0.166416,0.151589,0.095049,0.166416,0.041355,0.086854,0.113256,0.098430,0.041890,0.113256,0.011805,0.140014,0.053159,0.210117,0.195291,0.138751,0.210117,0.085056,0.043153,0.043702,0.096861,0.039016,0.024189,0.032350,0.039016,...,0.101029,0.102435,0.130848,0.165909,0.213594,0.213594,0.008621,0.023447,0.079987,0.008621,0.133682,0.261891,0.175036,0.121877,0.218738,0.047636,0.126726,0.130521,0.147347,0.221152,0.160091,0.076162,0.064230,0.115305,0.140508,0.144069,0.009418,0.138678,0.043492,0.063972,0.064415,0.077555,0.153872,0.121185,0.119779,0.091366,0.056305,0.008621,0.008621,0.222214
2,0.097006,0.099051,0.196056,0,0.097006,0.099051,0.197889,0.294895,0.098839,0.197889,0.323073,0.420079,0.224022,0.323073,0.125184,0.186627,0.283633,0.087576,0.186627,0.011262,0.136446,0.130447,0.227452,0.031396,0.130447,0.067443,0.192626,0.056181,0.218066,0.315072,0.119016,0.218066,0.020177,0.105007,0.031439,0.087620,0.038044,0.135050,0.061006,0.038044,...,0.000494,0.006360,0.006158,0.033650,0.127929,0.127929,0.043688,0.140693,0.055363,0.043688,0.154202,0.279385,0.142940,0.086759,0.174379,0.005643,0.102093,0.144752,0.174812,0.087941,0.074424,0.089609,0.003455,0.147541,0.054193,0.069555,0.084488,0.120292,0.035471,0.026014,0.075619,0.019626,0.117394,0.084735,0.090601,0.078083,0.117891,0.043688,0.043688,0.084241
3,0.024989,0.050223,0.075212,0,0.024989,0.050223,0.126632,0.151621,0.076409,0.126632,0.076875,0.101864,0.026652,0.076875,0.049757,0.156209,0.181198,0.105986,0.156209,0.029577,0.079334,0.114409,0.139399,0.064187,0.114409,0.012222,0.037534,0.041800,0.137521,0.162510,0.087298,0.137521,0.010889,0.060646,0.018688,0.023111,0.025548,0.000558,0.075770,0.025548,...,0.275644,0.318238,0.310798,0.261226,0.385062,0.385062,0.024101,0.049090,0.026121,0.024101,0.102531,0.052774,0.132108,0.090308,0.113420,0.049648,0.087579,0.126865,0.111339,0.166021,0.098746,0.102437,0.038643,0.076978,0.151718,0.021442,0.111813,0.082358,0.045167,0.046630,0.049152,0.004887,0.117996,0.085317,0.042723,0.050163,0.099735,0.024101,0.024101,0.360961
4,0.031143,0.061256,0.030114,0,0.031143,0.061256,0.048167,0.017024,0.013089,0.048167,0.134764,0.103621,0.073507,0.134764,0.086597,0.141375,0.110232,0.080119,0.141375,0.093208,0.006611,0.126412,0.095269,0.065156,0.126412,0.078245,0.008352,0.014963,0.195353,0.164210,0.134097,0.195353,0.147186,0.060590,0.053978,0.068941,0.030095,0.001048,0.031162,0.030095,...,0.133218,0.142490,0.107539,0.149699,0.224556,0.224556,0.024287,0.055429,0.085543,0.024287,0.072454,0.159050,0.165662,0.150699,0.219640,0.054381,0.171454,0.171289,0.169075,0.168031,0.223654,0.136651,0.120110,0.173127,0.261584,0.180769,0.076161,0.159417,0.056674,0.103605,0.145995,0.142238,0.250164,0.115625,0.106353,0.141304,0.099144,0.024287,0.024287,0.248843


In [ ]:
print(train.shape)

(150, 596)


In [ ]:
pca = PCA(whiten=True)
pca.fit(data)
variance = pd.DataFrame(pca.explained_variance_ratio_)
np.cumsum(pca.explained_variance_ratio_)

array([0.224933  , 0.34055914, 0.41194   , 0.47406611, 0.5260937 ,
       0.5634532 , 0.59881892, 0.63200694, 0.66215058, 0.6887922 ,
       0.71019499, 0.73016214, 0.74716108, 0.7630392 , 0.77814312,
       0.79256131, 0.80501491, 0.81709319, 0.82857371, 0.83846154,
       0.84797267, 0.85667559, 0.86500742, 0.87282312, 0.88008841,
       0.88682917, 0.89270303, 0.89848133, 0.90399686, 0.90902141,
       0.9139036 , 0.91815655, 0.9222748 , 0.92612756, 0.92990596,
       0.93321069, 0.93640118, 0.93955865, 0.94254379, 0.94529901,
       0.94791718, 0.95033645, 0.95258321, 0.95462654, 0.95656349,
       0.95828346, 0.95995984, 0.961577  , 0.96304941, 0.96437739,
       0.96568436, 0.96695246, 0.96817495, 0.96932601, 0.97040638,
       0.97143337, 0.97244976, 0.97336541, 0.97427072, 0.97510283,
       0.97589549, 0.97667951, 0.97743632, 0.97818272, 0.97890677,
       0.97959174, 0.98027202, 0.98093661, 0.98158054, 0.98219442,
       0.9827933 , 0.98337521, 0.98393243, 0.98446685, 0.98498

In [ ]:
pca = PCA(n_components=16,whiten=True)
pca = pca.fit(data)
dataPCA = pca.transform(data)
print(dataPCA.shape)
print(label.shape)

(150, 16)
(150, 596)


In [ ]:
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(dataPCA)
train = scaler.transform(dataPCA)


In [ ]:
from sklearn.multioutput import MultiOutputRegressor
reg = MultiOutputRegressor(SGDRegressor())
reg.fit(train, label)

MultiOutputRegressor(estimator=SGDRegressor(alpha=0.0001, average=False,
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.01, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='invscaling',
                                            loss='squared_loss', max_iter=1000,
                                            n_iter_no_change=5, penalty='l2',
                                            power_t=0.25, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                     n_jobs=None)

In [ ]:
reg.score(train, label)

0.6992226032462652

In [ ]:
# R2 Score

def lets_try(train,labels):
    results={}
    test = train[-15:]
    test_labels = labels[-15:]
    train = train[:-15]
    train_labels = labels[:-15]
    print(train.shape)
    print(test.shape)
    print(train_labels.shape)
    print(test_labels.shape)
    def test_model(clf):
        
        # cv = KFold(n_splits=5,shuffle=True,random_state=45)
        # r2 = make_scorer(mean_squared_error)
        # r2_val_score = cross_val_score(clf, train, labels, cv=cv,scoring=r2)
        # scores=[r2_val_score.mean()]
        
        clf.fit(train, train_labels)
        test_predict = clf.predict(test)
        scores = mean_squared_error(test_labels, test_predict)

        return [scores]

    clf = linear_model.LinearRegression()
    print(f"Linear Regression = {test_model(clf)}")
    
    clf = linear_model.Ridge()
    print(f"Ridge = {test_model(clf)}")
    
    clf = MultiOutputRegressor(SGDRegressor())
    print(f"SGDRegressor = {test_model(clf)}")

    clf = MultiOutputRegressor(linear_model.BayesianRidge())
    print(f"BayesianRidge = {test_model(clf)}")
    
    clf = MultiOutputRegressor(linear_model.HuberRegressor())
    print(f"HuberRegressor= {test_model(clf)}")
    
    clf = linear_model.Lasso(alpha=1e-4)
    print(f"Lasso = {test_model(clf)}")
        
    clf = BaggingRegressor()
    print(f"BaggingRegressor = {test_model(clf)}")

    clf = ElasticNet()
    print(f"ElasticNet = {test_model(clf)}")
    
    clf = RandomForestRegressor()
    print(f"RandomForestRegressor= {test_model(clf)}")
    
    clf = MultiOutputRegressor(AdaBoostRegressor())
    print(f"AdaBoostRegressor = {test_model(clf)}")
    
    clf = MultiOutputRegressor(svm.SVR())
    print(f"SVM RBF = {test_model(clf)}")
    
    clf = MultiOutputRegressor(svm.SVR(kernel="linear"))
    print(f"SVM Linear = {test_model(clf)}")
       
    clf = MultiOutputRegressor(svm.SVR(kernel="rbf"))
    print(f"SVM Linear rbf {test_model(clf)}")
    
    # results = pd.DataFrame.from_dict(results,orient='index')
    # results.columns=["MSE"] 
    # results=results.sort(columns=["MSE"],ascending=False)
    # results.plot(kind="bar",title="Model Scores")
    # axes = plt.gca()
    # axes.set_ylim([0.5,1])
    # return results

# lets_try(train,train_labels)
# print(train_labels.shape)
lets_try(, label)

(135, 16)
(15, 16)
(135, 596)
(15, 596)
Linear Regression = [8.70361583403411]
Ridge = [8.704807448317013]
SGDRegressor = [8.711770974619624]
BayesianRidge = [9.471460483134171]
HuberRegressor= [8.70897055428651]
Lasso = [8.703610077548264]
BaggingRegressor = [8.572025807131237]
BaggingRegressor = [8.930873038434518]
ElasticNet = [8.930426473499065]
RandomForestRegressor= [9.275201109403072]


KeyboardInterrupt: ignored

In [ ]:
import xgboost
clf = BaggingRegressor()
clf.fit(data, label)


BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=10,
                 n_jobs=None, oob_score=False, random_state=None, verbose=0,
                 warm_start=False)

In [ ]:
test = pd.read_csv('test_t0.csv')
test = test.drop("ID",1)
print(test.shape)
test.head()

(80, 595)


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f556,f557,f558,f559,f560,f561,f562,f563,f564,f565,f566,f567,f568,f569,f570,f571,f572,f573,f574,f575,f576,f577,f578,f579,f580,f581,f582,f583,f584,f585,f586,f587,f588,f589,f590,f591,f592,f593,f594,f595
0,0.122661,0.109112,0.231773,0,0.122661,0.109112,0.092518,0.215179,0.016593,0.092518,0.145721,0.268382,0.036609,0.145721,0.053202,0.207891,0.330552,0.098779,0.207891,0.115373,0.062170,0.105800,0.228461,0.003311,0.105800,0.013282,0.039920,0.102091,0.136860,0.259521,0.027748,0.136860,0.044342,0.008861,0.071031,0.031060,0.018215,0.140876,0.090897,0.018215,...,0.047609,0.112625,0.125787,0.124356,0.215772,0.215772,0.004597,0.127258,0.104515,0.004597,0.087922,0.141124,0.203295,0.101204,0.132264,0.013619,0.176267,0.120791,0.196560,0.163665,0.207902,0.065500,0.001600,0.026432,0.187328,0.026848,0.093334,0.115236,0.047054,0.052192,0.083222,0.155292,0.176474,0.163567,0.098551,0.085389,0.086820,0.004597,0.004597,0.211176
1,0.045889,0.077398,0.031509,0,0.045889,0.077398,0.181591,0.135702,0.104193,0.181591,0.079100,0.033211,0.001702,0.079100,0.102491,0.109561,0.063672,0.032163,0.109561,0.072030,0.030461,0.104044,0.058154,0.026646,0.104044,0.077547,0.024944,0.005517,0.181465,0.135576,0.104067,0.181465,0.000126,0.102365,0.071904,0.077421,0.002626,0.043263,0.074772,0.002626,...,0.056347,0.081727,0.075073,0.080828,0.177357,0.177357,0.002731,0.048621,0.080129,0.002731,0.184322,0.081831,0.112292,0.106775,0.184197,0.005358,0.152657,0.110212,0.131043,0.210307,0.115340,0.139567,0.080000,0.131111,0.134455,0.058544,0.080568,0.159891,0.025726,0.076265,0.087444,0.107125,0.168047,0.123742,0.098362,0.105015,0.099260,0.002731,0.002731,0.180088
2,0.020038,0.026596,0.046635,0,0.020038,0.026596,0.114418,0.134456,0.087822,0.114418,0.180229,0.200267,0.153633,0.180229,0.065811,0.177262,0.197300,0.150666,0.177262,0.062844,0.002967,0.136511,0.156549,0.109914,0.136511,0.022092,0.043718,0.040751,0.244241,0.264279,0.217645,0.244241,0.129823,0.064012,0.066979,0.107731,0.007501,0.012537,0.034097,0.007501,...,0.139746,0.083844,0.133137,0.168949,0.236086,0.236086,0.090372,0.070334,0.116968,0.090372,0.204790,0.270601,0.267634,0.226883,0.334613,0.082871,0.265359,0.247751,0.239643,0.292382,0.241814,0.268363,0.193135,0.160032,0.205748,0.116001,0.050774,0.219221,0.058565,0.132854,0.189319,0.125778,0.253501,0.186712,0.242613,0.193321,0.157509,0.090372,0.090372,0.326458
3,0.015838,0.073108,0.057270,0,0.015838,0.073108,0.112882,0.097044,0.039774,0.112882,0.281027,0.265189,0.207919,0.281027,0.168146,0.077977,0.062139,0.004869,0.077977,0.034905,0.203050,0.123877,0.108039,0.050769,0.123877,0.010995,0.157150,0.045900,0.151164,0.135326,0.078056,0.151164,0.038282,0.129863,0.073187,0.027287,0.014713,0.001125,0.058395,0.014713,...,0.135869,0.142534,0.112490,0.172141,0.237667,0.237667,0.002926,0.012912,0.070182,0.002926,0.109955,0.278101,0.075051,0.120951,0.148238,0.011787,0.148583,0.113695,0.114933,0.205219,0.170457,0.113402,0.075274,0.058549,0.129987,0.145470,0.050776,0.101857,0.031142,0.069858,0.072407,0.051349,0.140081,0.098872,0.092207,0.122251,0.062600,0.002926,0.002926,0.234741
4,0.007235,0.048022,0.040787,0,0.007235,0.048022,0.135871,0.128635,0.087848,0.135871,0.314094,0.306859,0.266072,0.314094,0.178223,0.118140,0.110905,0.070118,0.118140,0.017730,0.195954,0.093777,0.086542,0.045755,0.093777,0.042093,0.220317,0.024363,0.204410,0.197175,0.156388,0.204410,0.068540,0.109684,0.086270,0.110633,0.017383,0.024618,0.065405,0.017383,...,0.073855,0.081441,0.093111,0.152142,0.215760,0.215760,0.002288,0.004948,0.045735,0.002288,0.133583,0.311806,0.115853,0.091490,0.202123,0.019670,0.119562,0.102787,0.090560,0.267360,0.176748,0.144538,0.066852,0.085370,0.093304,0.139875,0.129553,0.138968,0.051438,0.047755,0.057658,0.070526,0.123818,0.139617,0.132032,0.120361,0.061331,0.002288,0.002288,0.213473


In [ ]:
test = pca.transform(test)
prediction = clf.predict(test)

ValueError: ignored

In [ ]:
print(prediction.shape)
df = pd.DataFrame(prediction)
df.head()

In [ ]:
;batu_flat = df.values.flatten()
print(batu_flat.shape)

In [ ]:
meltedDF = df.to_numpy().flatten()
print(meltedDF.shape)

(47600,)


In [ ]:
anan = pd.DataFrame(meltedDF)
anan.head()

,0
0,0.061191
1,0.069773
2,0.100056
3,0.000000
4,0.061191


In [ ]:
# anan.insert(0, "ID", np.arange(1, anan.shape[0]+1))

In [ ]:
anan.head()

,0
0,0.061191
1,0.069773
2,0.100056
3,0.000000
4,0.061191


In [ ]:
anan.tail()

,0
47595,0.134184
47596,0.134456
47597,0.037313
47598,0.037313
47599,0.310460


In [ ]:
print(anan.shape)

(47600, 1)


In [ ]:
baban = anan.rename(columns={0: "predicted"})

In [ ]:
baban.head()

,predicted
0,0.061191
1,0.069773
2,0.100056
3,0.000000
4,0.061191


In [ ]:
csvdosyasiseysi = baban.to_csv("test_results2.csv")

In [ ]:
 from xgboost import XGBRFRegressor
 model = MultiOutputRegressor(XGBRFRegressor(objective='reg:squarederror', n_estimators=100, verbosity=0, subsmaple=0.9, colsample_bynode=0.2))

NameError: ignored

In [ ]:
model.fit(dataPCA, label)
prediction = model.predict(dataPCA)
scores = mean_squared_error(prediction, label)
print(scores)

0.002625866532645193


In [ ]:
test = pd.read_csv('test_t0.csv')
test = test.drop("ID",1)
print(test.shape)
prediction = model.predict(pca.transform(test))

(80, 595)


In [ ]:
print(prediction.shape)

(80, 595)


In [ ]:
prediction2 = pd.DataFrame(prediction.flatten())
prediction2.shape

(47600, 1)

In [ ]:
pre3 = prediction2.rename(columns={0:"predicted"})

In [ ]:
pre3.index.name = "ID"
print(pre3.shape)
pre3.to_csv("deneme.csv")

(47600, 1)


In [ ]:
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

In [ ]:
n_scores = cross_val_score(model, dataPCA, label, scoring='neg_mean_squared_error', cv=cv)

In [ ]:
model.fit(dataPCA, label)

In [ ]:
print(n_scores)

In [ ]:
prediction.shape == (80, 595)

True

In [ ]:
make_submission(prediction, "deneme2.csv")

deneme2.csv saved at /content


,predicted
ID,
0,0.048186
1,0.084227
2,0.076308
3,0.004137
4,0.048186
...,...
47595,0.129521
47596,0.124411
47597,0.039825


### KFold = n_split=10 results
(135, 19)
(15, 19)
(135, 595)
(15, 595)
Linear Regression = [0.005802272956797038]
Ridge = [0.005799988728181303]
BayesianRidge = [0.004297786222005502]
HuberRegressor= [0.004622051656589287]
Lasso = [0.005790250820408533]
BaggingRegressor = [0.004056072586337191]
ElasticNet = [0.0030826790406581098]
RandomForestRegressor= [0.00402007929036715]
AdaBoostRegressor = [0.0041291666017797945]
SVM RBF = [0.0054829324623424285]
SVM Linear = [0.007069600173376448]
SVM Linear rbf [0.0054829324623424285]
(135, 16)
(15, 16)
(135, 595)
(15, 595)
Linear Regression = [0.006744191885778283]
Ridge = [0.006743680477484871]
BayesianRidge = [0.006507214061402251]
HuberRegressor= [0.006800372756951189]
Lasso = [0.006742680791274501]
BaggingRegressor = [0.008102009029265259]
ElasticNet = [0.0069362924667603545]
RandomForestRegressor= [0.010728668527239145]
AdaBoostRegressor = [0.009640671034316697]
SVM RBF = [0.007188838358384941]
SVM Linear = [0.007198396567356369]
SVM Linear rbf [0.007188838358384941]
(135, 16)
(15, 16)
(135, 595)
(15, 595)
Linear Regression = [0.0023045183308133764]
Ridge = [0.0023041497995984583]
BayesianRidge = [0.0021378877788895876]
HuberRegressor= [0.0021207545646454214]
Lasso = [0.002302697452722897]
BaggingRegressor = [0.002598639753050235]
ElasticNet = [0.0025102862420139298]
RandomForestRegressor= [0.002242842157217008]
AdaBoostRegressor = [0.0023012304887634768]
SVM RBF = [0.004641813739452005]
SVM Linear = [0.00367907916418258]
SVM Linear rbf [0.004641813739452005]
(135, 16)
(15, 16)
(135, 595)
(15, 595)
Linear Regression = [0.0025639376342007263]
Ridge = [0.002563666511163113]
BayesianRidge = [0.0024434805608064642]
HuberRegressor= [0.002468197582583029]
Lasso = [0.0025624369470381076]
BaggingRegressor = [0.002487186469989797]
ElasticNet = [0.002904297259787476]
RandomForestRegressor= [0.002333295561098554]
AdaBoostRegressor = [0.0025187103614842717]
SVM RBF = [0.004504221270852039]
SVM Linear = [0.0038039811351258303]
SVM Linear rbf [0.004504221270852039]
(135, 16)
(15, 16)
(135, 595)
(15, 595)
Linear Regression = [0.005164427021598518]
Ridge = [0.005164167810874289]
BayesianRidge = [0.005153677465195344]
HuberRegressor= [0.005226145178489968]
Lasso = [0.005164081922971847]
BaggingRegressor = [0.005703330385007766]
ElasticNet = [0.005776121174494068]
RandomForestRegressor= [0.005401241834928647]
AdaBoostRegressor = [0.005330574257363599]
SVM RBF = [0.006677056534854889]
SVM Linear = [0.0062700324730901634]
SVM Linear rbf [0.006677056534854889]
(135, 16)
(15, 16)
(135, 595)
(15, 595)
Linear Regression = [0.0020036552371676883]
Ridge = [0.0020033225996146883]
BayesianRidge = [0.0018156061980202094]
HuberRegressor= [0.0017102318993381855]
Lasso = [0.0020020438619769215]
BaggingRegressor = [0.0018897397444012892]
ElasticNet = [0.002194761461803483]
RandomForestRegressor= [0.0017733194673465297]
AdaBoostRegressor = [0.0019269472979016888]
SVM RBF = [0.0038769952379162417]
SVM Linear = [0.0033247080027624287]
SVM Linear rbf [0.0038769952379162417]
(135, 15)
(15, 15)
(135, 595)
(15, 595)
Linear Regression = [0.002786927428653145]
Ridge = [0.002786718118785138]
BayesianRidge = [0.0026657877284159037]
HuberRegressor= [0.00269085722337065]
Lasso = [0.0027858243055081905]
BaggingRegressor = [0.002955599320451397]
ElasticNet = [0.0028839527435591213]
RandomForestRegressor= [0.002702839123732035]
AdaBoostRegressor = [0.002768456849174212]
SVM RBF = [0.004920677326775243]
SVM Linear = [0.0038845834542473538]
SVM Linear rbf [0.004920677326775243]
(135, 16)
(15, 16)
(135, 595)
(15, 595)
Linear Regression = [0.0052766083802658495]
Ridge = [0.005275847989638481]
BayesianRidge = [0.004820034635472762]
HuberRegressor= [0.005026180298915299]
Lasso = [0.005273642590599138]
BaggingRegressor = [0.004758714351386203]
ElasticNet = [0.004821847827563947]
RandomForestRegressor= [0.0045268301422334905]
AdaBoostRegressor = [0.0046410166751561805]
SVM RBF = [0.005492814286647762]
SVM Linear = [0.00594884576679866]
SVM Linear rbf [0.005492814286647762]
(135, 16)
(15, 16)
(135, 595)
(15, 595)
Linear Regression = [0.010119589993062863]
Ridge = [0.010119753227389148]
BayesianRidge = [0.010289064935715768]
HuberRegressor= [0.010196346157144919]
Lasso = [0.010120091934235345]
BaggingRegressor = [0.010162610548886604]
ElasticNet = [0.011354407955110935]
RandomForestRegressor= [0.009976735097034919]
AdaBoostRegressor = [0.010271722899001794]
SVM RBF = [0.011318486537443332]
SVM Linear = [0.011190860576632366]
SVM Linear rbf [0.011318486537443332]
(135, 16)
(15, 16)
(135, 595)
(15, 595)
Linear Regression = [0.002596325787652625]
Ridge = [0.0025961320992643933]
BayesianRidge = [0.0025376615663518782]
HuberRegressor= [0.002633996183132533]
Lasso = [0.0025950992079001494]
BaggingRegressor = [0.002507620790240666]
ElasticNet = [0.0029951336319722984]
RandomForestRegressor= [0.002490842896038554]
AdaBoostRegressor = [0.0026133376407376676]
SVM RBF = [0.004267637341080629]
SVM Linear = [0.004026512971278571]
SVM Linear rbf [0.004267637341080629]

In [246]:
anan = {
    'a':[1,2],
    'b':[3,4]
}
baban = {
    'a':[3],
    'b':[5]
}

In [247]:
anan['c'] = [5]

In [223]:
anandf = pd.DataFrame(anan).T
print(anandf)

   0  1
a  1  2
b  3  4


In [224]:
anandf.insert(2, "Var", anandf.var(axis=1))
anandf.insert(2, "Mean", anandf.mean(axis=1))
print(anandf)

   0  1      Mean  Var
a  1  2  1.166667  0.5
b  3  4  2.500000  0.5


In [167]:
anandf.append(3)

TypeError: ignored

In [176]:
anan = {'a':[]}
def update(dictionariy):
    dictionariy['a'].append(3)
update(anan)
print(anan)
update(anan)
print(anan)

{'a': [3]}
{'a': [3, 3]}


In [179]:
clf = MultiOutputRegressor(AdaBoostRegressor())

In [188]:
type(clf.estimator).__name__

'AdaBoostRegressor'

In [193]:
'a' in anan.keys()

True